In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
from IPython.core.debugger import Pdb
import numpy as np
import math
import sys
import os
import time
import subprocess
import pickle
sys.path.append('../../')
print(sys.path)
import torch
import torch.nn as nn
import torch.nn.functional as F
import librosa
from torch.utils.data import Dataset, DataLoader
from torch.nn.parameter import Parameter
from torch.autograd import Variable
import matplotlib.pyplot as plt
from models.mlmodeldic import MLModelDic
from models.phoneme import Phoneme43
from tqdm import tqdm
from mlutils.utils import time_elapse, time_since, update_lr, shape, calc_acc, plot
from sklearn.model_selection import train_test_split

['', '/home/kaz/miniconda3/envs/core/lib/python36.zip', '/home/kaz/miniconda3/envs/core/lib/python3.6', '/home/kaz/miniconda3/envs/core/lib/python3.6/lib-dynload', '/home/kaz/miniconda3/envs/core/lib/python3.6/site-packages', '/home/kaz/miniconda3/envs/core/lib/python3.6/site-packages/warpctc_pytorch-0.1-py3.6-linux-x86_64.egg', '/home/kaz/miniconda3/envs/core/lib/python3.6/site-packages/setuptools-27.2.0-py3.6.egg', '/home/kaz/miniconda3/envs/core/lib/python3.6/site-packages/torchvision-0.2.1-py3.6.egg', '/home/kaz/miniconda3/envs/core/lib/python3.6/site-packages/IPython/extensions', '/home/kaz/.ipython', '../../']


In [2]:
class FFTNetQueue(object):
    def __init__(self, batch_size, n_channels, input_size, is_cuda=False):
        super(FFTNetQueue, self).__init__()
        self.__batch_size__ = batch_size
        self.__n_channels__ = n_channels
        self.__input_size__ = input_size
        self.__is_cuda__ = is_cuda
        self.queue = []
        self.reset(batch_size, is_cuda)
    
    def reset(self, batch_size, is_cuda=False):
        self.queue = torch.zeros([batch_size, self.__n_channels__, self.__input_size__])
        if is_cuda:
            self.queue = self.queue.cuda()
            
    def enqueue(self, sample_to_push):
        """Return the last sample and insert the new sample to the end of queue.
        
        args:
            - sample_to_push: (B, C, T=1)
        
        1. remove first, 2. push sample_to_push to last, 3. return z
        [0, 1, 2, 3, .., y, z] -> [1, 2, 3, .., y, z, sample_to_push]
        """
        Pdb().set_trace()
        sample_to_pop = self.queue[:, :, -1:].data  # get last
        self.queue[:, :, :-1] = self.queue[:, :, 1:]  # [0, 1, 2, 3, .., y, z] -> [1, 2, 3, .., y, z, z]
        return sample_to_pop  # return the last

In [3]:
class FFTLayer(nn.Module):
    def __init__(self, in_channels: int, out_channels: int, layer_idx:int, input_size: int):
        super().__init__()
        # settings
        self.__in_channels__ = in_channels
        self.__out_channels__ = out_channels
        #self.__f0_channels__ = f0_channels  # always 1 because only the first layer
        #self.__phonemes_channels__ = phonemes_channels   # always 1 because only the first layer
        self.__input_size__ = input_size  # input size for this layer (=depth**2)
        self.__layer_idx__ = layer_idx
        # generate queues
        self.sample_queue = None
        self.f0_queue = None
        self.phonemes_queue = None
        #self.f0_buffer = None
        #self.phonemes_buffer = None
        # layers
        self.conv1d_L = nn.Conv1d(in_channels, out_channels, kernel_size=1)
        self.conv1d_R = nn.Conv1d(in_channels, out_channels, kernel_size=1)
        if layer_idx == 0:
            self.conv1d_f0_L = nn.Conv1d(1, out_channels, kernel_size=1)  #nn.Conv1d(f0_channels, out_channels, kernel_size=1)
            self.conv1d_f0_R = nn.Conv1d(1, out_channels, kernel_size=1)  #nn.Conv1d(f0_channels, out_channels, kernel_size=1)
            self.conv1d_phonemes_L = nn.Conv1d(1, out_channels, kernel_size=1)  #nn.Conv1d(phonemes_channels, out_channels, kernel_size=1)
            self.conv1d_phonemes_R = nn.Conv1d(1, out_channels, kernel_size=1)  #nn.Conv1d(phonemes_channels, out_channels, kernel_size=1)
        self.conv1d_out = nn.Conv1d(out_channels, out_channels, kernel_size=1)
    
    def forward(self, x, f0=None, phonemes=None):
        #Pdb().set_trace()
        x_L = self.conv1d_L(x[:, :, :-self.__input_size__//2])
        x_R = self.conv1d_R(x[:, :, self.__input_size__//2:])
        if f0 is not None and phonemes is not None:
            # only called in the first layer
            f0_L = self.conv1d_f0_L(f0[:, :, :-self.__input_size__//2])
            f0_R = self.conv1d_f0_R(f0[:, :, self.__input_size__//2:])
            # NOTE: categorical sequencial data works for convnets, but works without any spatial encoding?
            phonemes_L = self.conv1d_phonemes_L(phonemes[:, :, :-self.__input_size__//2])
            phonemes_R = self.conv1d_phonemes_R(phonemes[:, :, self.__input_size__//2:])
            z = x_L + x_R + f0_L + f0_R + phonemes_L + phonemes_R
        else:
            z = x_L + x_R
        x = F.relu(z)
        return F.relu(self.conv1d_out(x))
    
    def generate_step(self, x, f0=None, phoneme=None):
        # 最初のstepで入力されるのはx=randint(1,256,(1,1,1))
        #Pdb().set_trace()
        B = x.shape[0]
        
        # init queues
        if self.sample_queue is None:
            self.sample_queue = FFTNetQueue(B, self.__in_channels__, self.__input_size__, x.is_cuda)
        if (self.f0_queue is None and f0 is not None) and (self.phonemes_queue is None and phoneme is not None):
            self.f0_queue = FFTNetQueue(B, self.__in_channels__, self.__input_size__, x.is_cuda)
            self.phonemes_queue = FFTNetQueue(B, self.__in_channels__, self.__input_size__, x.is_cuda)
            
        # input - current sample (L) / previous sample (R)
        x_L = x
        x_R = self.sample_queue.enqueue(x)  # (B, T+C) (enqueueされるのはbufferの末の要素のみ)
        # forward to conv1d
        z1 = self.conv1d_L(x_L)
        z2 = self.conv1d_R(x_R)
        z = z1 + z2
        
        # f0, phonemes
        if f0 is not None and phoneme is not None:
            f0 = torch.FloatTensor([f0]).view(1,1,1)
            phoneme = torch.FloatTensor([phoneme]).view(1,1,1)
            f0_L = f0
            f0_R = self.f0_queue.enqueue(f0)
            phoneme_L = phoneme
            phoneme_R = self.phonemes_queue.enqueue(phoneme)
            # forward to conv1d
            z_f0_1 = self.conv1d_f0_L(f0_L)
            z_f0_2 = self.conv1d_f0_R(f0_R)
            z_phoneme_1 = self.conv1d_phonemes_L(phoneme_L)
            z_phoneme_2 = self.conv1d_phonemes_R(phoneme_R)
            z = z + z_f0_1 + z_f0_2 + z_phoneme_1 + z_phoneme_2
            
        z = F.relu(z)
        z = F.relu(self.conv1d_out(z))
        z = z.view(B, -1, 1)
        return z  # (B, 1, 1)

    #def generate_step(self, x, f0=None, phonemes=None):
    #    """
    #    >>> x
    #    tensor([-1.0000, -1.0000, -1.0000,  0.1000,  0.2000,  0.3000,  0.4000,  0.5000,
    #             0.6000,  0.7000,  0.8000])
    #    >>> F.relu(x)
    #    tensor([0.0000, 0.0000, 0.0000, 0.1000, 0.2000, 0.3000, 0.4000, 0.5000, 0.6000,
    #            0.7000, 0.8000])
    #    >>> conv = torch.nn.Conv1d(len(x), out_channels=1, kernel_size=1)
    #    >>> conv(x.view(1,-1,1))
    #    tensor([[[0.4139]]], grad_fn=<SqueezeBackward1>)
    #    >>> F.relu(conv(x.view(1,-1,1)))
    #    tensor([[[0.4139]]], grad_fn=<ReluBackward>)
    #    """
    #    Pdb().set_trace()
    #    # MEMO: why only the first and the last?
    #    # mozillaの実装ではLにx.view(:, -1) (B, T=1), Rにbufferの末要素(B, T=1)を入力している
    #    x_L = self.conv1d_L(x[:, :, :1])  # x(t=0): (1, 1, 1025), x[:, :, :1](t=0) (1,1,1), x_L(t=0): (1, 256, 1)
    #    x_R = self.conv1d_R(x[:, :, -1:])  # x(t=0): (1, 1, 1025), x[:, :, :1](t=0) (1,1,1), x_R(t=0): (1, 256, 1)
#
    #    if f0 and phonemes:
    #        # list to tensor (B:1, C:1, T:len(f0))
    #        f0 = torch.FloatTensor(f0).view(1, 1, -1)  # f0 (1, 1, 16000) when seq_len is 16000
    #        phonemes = torch.FloatTensor(phonemes).view(1, 1, -1) 
    #        # forward
    #        f0_L = self.conv1d_f0_L(f0[:, :, :-self.__input_size__//2])  # f0[:, :, :-self.__input_size__//2] (1, 1, 14976), f0_L: (1, 256, 14976)  when seq_len is 16000
    #        f0_R = self.conv1d_f0_R(f0[:, :, self.__input_size__//2:])  # f0[:, :, self.__input_size__//2:] (1, 1, 14976), f0_R: (1, 256, 14976)  when seq_len is 16000
    #        # NOTE: categorical sequencial data works for convnets, but works without any spatial encoding?
    #        phonemes_L = self.conv1d_phonemes_L(phonemes[:, :, :-self.__input_size__//2])
    #        phonemes_R = self.conv1d_phonemes_R(phonemes[:, :, self.__input_size__//2:])
    #        x = x_L + x_R + f0_L + f0_R + phonemes_L + phonemes_R  # x_L: (1, 256, 1) x_R: (1, 256, 1)
    #    else:
    #        x = x_L + x_R
    #        
    #    # NOTE: t=0でbuffer size(1, 256, 513)と一致しているはず?
    #    x = F.relu(x)  # x(t=0): (1, 256, 14976) when seq_len is 16000
    #    x = F.relu(self.conv1d_out(x))  # x(t=0): (1, 256, 14976) when seq_len is 16000
    #    return x

In [4]:
class FFTNet(nn.Module):
    def __init__(self, n_channels=256, n_depth=11, n_classes=256, disable_cuda=False):
        super().__init__()
        self.__disable_cuda__ = disable_cuda
        self.__n_channels__ = n_channels
        #self.__f0_channels__ = f0_channels
        #self.__phonemes_channels__ = phonemes_channels
        self.__n_depth__ = n_depth
        self.__n_classes__ = n_classes  # 256 categorical μ-law encoding
        self.__receptive_field_size__ = 2**n_depth  # 2**11 = 2048
        fft_layers = []
        for idx, N in enumerate([2**i for i in range(n_depth, 0, -1)]):
            if idx == 0:
                fft_layers += [FFTLayer(1, n_channels, idx, N)]
            else:
                fft_layers += [FFTLayer(n_channels, n_channels, idx, N)]
        self.fft_layers = nn.ModuleList(fft_layers)
        self.fully_connected = nn.Linear(n_channels, n_classes)
        print(f'Receptive Field: {self.__receptive_field_size__} samples')
        self.num_params(self)
        
    def settings(self) -> dict:
        return {
            'n_channels': self.__n_channels__,
            'n_depth': self.__n_depth__,
            'n_classes': self.__n_classes__,
            'receptive_field_size': self.__receptive_field_size__
        }
    
    def settings_str(self) -> str:
        return f'channels{self.__n_channels__}_depth{self.__n_depth__}_class{self.__n_classes__}_rfsize{self.__receptive_field_size__}'
        
    def num_params(self, model) :
        parameters = filter(lambda p: p.requires_grad, model.parameters())
        parameters = sum([np.prod(p.size()) for p in parameters]) / 1_000_000
        print('Trainable Parameters: %.3f million' % parameters)
        
    def forward(self, x, f0=None, phonemes=None):
        #Pdb().set_trace()
        B, C, T = x.size()
        padding = torch.zeros(B, C, self.__receptive_field_size__)
        if not self.__disable_cuda__:
            padding = padding.cuda()
        x = torch.cat([padding, x], dim=-1)
        f0 = torch.cat([padding, f0], dim=-1)
        phonemes = torch.cat([padding, phonemes], dim=-1)
        _x = x
        for i, fft_layer in enumerate(self.fft_layers):
            if i == 0:
                _x = fft_layer(_x, f0=f0, phonemes=phonemes)
            else:
                _x = fft_layer(_x)
        _x = self.fully_connected(_x.transpose(1, 2))
        return F.log_softmax(_x, dim=-1)
    
    def padding(self, x):
        B, C, T = x.size()
        max_len = np.max([d.shape[0] - 1 for d in x])
        x_pad = np.zeros([B, self.__receptive_field_size__ + max_len])
        for i, signal in enumerate(x):
            pad_w = max_len - signal.shape[0]
        # pad left with receptive field and right with max_len in the batch
        wav = np.pad(wav, [self.__receptive_field_size__ - 1, pad_w], 
                        mode='constant', constant_values=0.0)
        mel = np.pad(mel, [[self.__receptive_field_size__ - 1, pad_w], [0, 0]], 
                        mode='constant', constant_values=0.0)
        
    # queues are initialized in the first generate_step in each layer if queue is None
    #def init_queues(self, batch_size):
    #    for i, f in enumerate(self.fft_layers):
    #        # init each layers queue with zeros(B, C, field size)
    #        f.init_queue(batch_size, self.__disable_cuda__)
        
    #def init_buffer(self):
    #    """Initialize weight buffer for each layer.
    #    
    #    set [
    #        rand(1, 1, input size of layer 0),
    #        rand(1, 1, input size of layer 1),
    #        ...
    #    ] to self.buffer
    #    
    #    
    #    ipdb> p self.signal_buffer[0].shape
    #    torch.Size([1, 1, 1025])
    #    ipdb> p self.signal_buffer[1].shape
    #    torch.Size([1, 256, 513])
    #    ipdb> p self.signal_buffer[2].shape
    #    torch.Size([1, 256, 257])
    #    ipdb> p self.signal_buffer[3].shape
    #    torch.Size([1, 256, 129])
    #    ipdb> p self.signal_buffer[4].shape
    #    torch.Size([1, 256, 65])
    #    ipdb> p self.signal_buffer[5].shape
    #    torch.Size([1, 256, 33])
    #    ipdb> p self.signal_buffer[10].shape
    #    torch.Size([1, 256, 2])
    #    ipdb> p self.signal_buffer[11].shape
    #    torch.Size([1, 256, 1])
    #    """
    #    Pdb().set_trace()
    #    # MEMO: mozillaの実装では各レイヤーインスタンスが2次元のbuffer(B, T)をもつ.
    #    # 長さ1の入力サンプルを末に連結しつつその前のstepの入力サンプルをpopする
    #    x = torch.rand(1, 1, self.fft_layers[0].__input_size__)
    #    f0 = torch.rand(1, 1, self.fft_layers[0].__input_size__)
    #    phonemes = torch.rand(1, 1, self.fft_layers[0].__input_size__)
    #    if not self.__disable_cuda__:
    #        x = x.cuda()
    #        f0 = f0.cuda()
    #        phonemes = phonemes.cuda()
    #    #self.signal_buffer = [x[:, :, self.fft_layers[0].__input_size__//2-1:]]  # self.signal_buffer[0]: (1, 1, 1025) <= 2048-(2048//2-1) = 2048-1023 = 1025 
    #    self.signal_buffer = [x[:, :, self.fft_layers[0].__input_size__//2:]]  # half size of input field
    #    for i, f in enumerate(self.fft_layers):
    #        x_L = f.conv1d_L(x[:, :, :-f.__input_size__//2])  # x_L(i=0): (1, 1, 1024), x_L(i=1): (1, 256, 512)
    #        x_R = f.conv1d_R(x[:, :, f.__input_size__//2:])  # x_R(i=0): (1, 1, 1024), x_R(i=1): (1, 256, 512)
    #        x = x_L + x_R  # z(i=0): (1, 256, 1024) = x_L: (1, 256, 1024) + x_R: (1, 256, 1024), z(i=1): (1, 256, 512)
    #        x = F.relu(x)  # z(i=0): (1, 256, 1024), z(i=1): (1, 256, 512)
    #        x = F.relu(f.conv1d_out(x))  # z(i=0): (1, 256, 1024), z(i=1): (1, 256, 512)
    #        self.signal_buffer += [x[:, :, f.__input_size__//4-1:]]  #  z[:, :, f.__input_size__//4-1:] (i=0) (1, 256, 513), z[:, :, f.__input_size__//4-1:] (i=1) (1, 256, 257)
            
    def push_to_signal_buffer(self, i, y):
        self.signal_buffer[i] = torch.cat([self.signal_buffer[i], y], dim=-1)[:, :, 1:]

    def push_to_f0_buffer(self, i, y):
        self.f0_buffer[i] = torch.cat([self.f0_buffer[i], y], dim=-1)[:, :, 1:]

    def push_to_phonemes_buffer(self, i, y):
        self.phonemes_buffer[i] = torch.cat([self.phonemes_buffer[i], y], dim=-1)[:, :, 1:]
        
    def class2float(self, x) :
        """Convert 256 categorical class to -1.0 ~ 1.0 value.
        
        0 -> -1.0
        1 -> -0.9921568627450981
        2 -> -0.9843137254901961
        ...
        256 -> 1.0
        """
        return 2 * x.float() / (self.__n_classes__ - 1) - 1
    
    def duration_to_seq_len(self, duration_ms: int, sample_rate=16000):
        return int(16000/1000)*duration_ms
    
    def generate(self, duration_ms: int, phonemes=None, pitches=None, sample_rate=16000):
        #Pdb().set_trace()
        with torch.no_grad():
            c = 2
            B = 1
            output = []
            start = time.time()
            seq_len = self.duration_to_seq_len(duration_ms)
            for t in range(seq_len):
                if t == 0:
                    # start sequence with random input
                    x = torch.randint(1, 256, (1,1,1))
                for i, f in enumerate(self.fft_layers):
                    if i == 0 and (pitches and phonemes):
                        x = f.generate_step(x, f0=pitches[t], phoneme=phonemes[t])
                    else:
                        x = f.generate_step(x)
                x = self.fully_connected(x.squeeze(-1))
                posterior = F.softmax(c * x.view(-1), dim=0)
                # TODO: 2.3.2 Conditional sampling
                dist = torch.distributions.Categorical(posterior)
                sample = self.class2float(dist.sample())
                output.append(sample)
                speed = (t + 1) / (time.time() - start)
                print(f'generate class {dist.sample()} ({sample}): {t+1}/{seq_len}, Speed: {speed:.2f} samples/sec')
                x = torch.FloatTensor([dist.sample()]).view(B, 1, 1)
                #Pdb().set_trace()
        return torch.stack(output).cpu().numpy()

    #def generate(self, duration_ms: int, phonemes=[], pitches=[], sample_rate=16000):
    #    def duration_to_seq_len(duration_ms: int, sample_rate=16000):
    #        return int(16000/1000)*duration_ms
    #    Pdb().set_trace()
    #    with torch.no_grad():
    #        c = 2
    #        self.init_buffers()
    #        output = []
    #        start = time.time()
    #        seq_len = duration_to_seq_len(duration_ms)
    #        for t in range(seq_len):
    #            for i, f in enumerate(self.fft_layers):
    #                if i == 0:
    #                    x = f.generate_step(self.signal_buffer[i], f0=pitches, phonemes=phonemes)
    #                else:
    #                    x = f.generate_step(self.signal_buffer[i])
    #                self.push_to_signal_buffer(i + 1, x)
    #            Pdb().set_trace()
    #            x = self.fully_connected(x.squeeze(-1))
    #            posterior = F.softmax(c * x.view(-1), dim=0)
    #            # TODO: 2.3.2 Conditional sampling
    #            dist = torch.distributions.Categorical(posterior)
    #            sample = self.class2float(dist.sample())
    #            output.append(sample)
    #            self.push_to_signal_buffer(0, sample.view(1, 1, 1))
    #            speed = (t + 1) / (time.time() - start)
    #            print(f'Generating: {t+1}/{seq_len}, Speed: {speed:.2f} samples/sec')
    #    self.buffer = None
    #    return torch.stack(output).cpu().numpy()
    
    def save_model(self, save_model_path:str):
        try:
            torch.save(self.state_dict(), save_model_path)
            # torch.save(self, save_model_path)  # * this fails when data parallel
        except Exception as e:
            print(e)
            
    def load_model(self, model_file_path:str):
        try:
            self.load_state_dict(
                torch.load(model_file_path, map_location=lambda storage, loc: storage))
            # torch.load(model_file_path)  # * this fails if trained on multiple GPU. use state dict.
        except Exception as e:
            print(e)

In [ ]:
disable_cuda = True

# load model
#best_model_dic = MLModelDic.best_model('fftnet', key='loss_average', lowest=True)
best_model_dic = MLModelDic.findOne({'_id': 'mlmodeldic-8bf0f5f7-3524-4b49-877c-c1de97b83d46'})
#best_model_dic.remove_model()
settings = best_model_dic.settings
model_path = best_model_dic.save_model_path
print('best model settings: ',settings)
print('model path: ',model_path)
print('loss ave: ',best_model_dic.loss_average)
print('_id', best_model_dic._id)
print('created', best_model_dic.created)
model = FFTNet(n_channels=settings['n_channels'], n_depth=settings['n_depth'], n_classes=settings['n_classes'], disable_cuda=disable_cuda)
model.load_model(model_path)
model.eval()  # without droput
if not disable_cuda:
    model.cuda()
else:
    model.cpu()

best model settings:  {'lr': 0.0001, 'epoch': 1, 'iter': 7615, 'n_steps': 60000, 'seq_len': 5000, 'n_channels': 256, 'n_depth': 11, 'n_classes': 256, 'receptive_field_size': 2048}
model path:  /diskB/6/out/models/fftnet/channels256_depth11_class256_rfsize2048_lr0.0001_loss1.355_nsteps60000_seqlen5000_iter1800000_index19
loss ave:  1.3552132371803582
_id mlmodeldic-8bf0f5f7-3524-4b49-877c-c1de97b83d46
created 2018-12-14 01:30:29.604000
Receptive Field: 2048 samples
Trainable Parameters: 2.108 million


In [ ]:
sample_rate = 16000
duration_ms = 100
seq_len = int(sample_rate / (duration_ms/1000))
model.cpu()
# TODO: test data from dataset
#output = model.generate(duration_ms=duration_ms, phonemes=[1]*seq_len, pitches=[200]*seq_len, sample_rate=sample_rate)
output = model.generate(duration_ms=duration_ms, phonemes=None, pitches=None, sample_rate=sample_rate)

> <ipython-input-2-3190f5e55ef0>(26)enqueue()
     24         """
     25         Pdb().set_trace()
---> 26         sample_to_pop = self.queue[:, :, -1:].data  # get last
     27         self.queue[:, :, :-1] = self.queue[:, :, 1:]  # [0, 1, 2, 3, .., y, z] -> [1, 2, 3, .., y, z, z]
     28         return sample_to_pop  # return the last

ipdb> n
> <ipython-input-2-3190f5e55ef0>(27)enqueue()
     24         """
     25         Pdb().set_trace()
     26         sample_to_pop = self.queue[:, :, -1:].data  # get last
---> 27         self.queue[:, :, :-1] = self.queue[:, :, 1:]  # [0, 1, 2, 3, .., y, z] -> [1, 2, 3, .., y, z, z]
     28         return sample_to_pop  # return the last

ipdb> c
> <ipython-input-2-3190f5e55ef0>(26)enqueue()
     24         """
     25         Pdb().set_trace()
---> 26         sample_to_pop = self.queue[:, :, -1:].data  # get last
     27         self.queue[:, :, :-1] = self.queue[:, :, 1:]  # [0, 1, 2, 3, .., y, z] -> [1, 2, 3, .., y, z, z]
     28         re

In [ ]:
plot(output)

In [ ]:
model = FFTNet(n_channels=N_CHANNELS, disable_cuda=DISABLE_CUDA)
if not DISABLE_CUDA:
    model.cuda()
print(model)

In [ ]:
model = FFTNet(n_channels=N_CHANNELS, disable_cuda=DISABLE_CUDA)
if not DISABLE_CUDA:
    model.cuda()
print(model)

In [ ]:
model = FFTNet(n_channels=N_CHANNELS, disable_cuda=DISABLE_CUDA)
if not DISABLE_CUDA:
    model.cuda()
print(model)